In [1]:
import pandas as pd 
import numpy as np
import regex as re

#### Adding The Player Info

In [2]:
players_2020 = pd.read_csv("C:/Users/Titoran/Documents/Cleaned Players Data/Players_2020.csv", index_col = [0])
players_2021 = pd.read_csv("C:/Users/Titoran/Documents/Cleaned Players Data/Players_2021.csv", index_col = [0])
players_2022 = pd.read_csv("C:/Users/Titoran/Documents/Cleaned Players Data/Players_2022.csv", index_col = [0])

In [3]:
df_NER = pd.read_csv('Rumours_Sent.csv', index_col = [0] )

In [4]:
df_NER = df_NER.loc[df_NER["Score"] == "Positive",]
df_NER = df_NER.drop(['Score'], axis=1)

In [5]:
#Unlisting the players to then join them
#df_NER["Player"] = [str(''.join(map(str, l))) for l in df_NER['Player']]

In [6]:
#Maing sure just the player name is in the column
df_NER["Player"] = df_NER["Player"].apply(lambda x: x[2:-2])

In [7]:
df2020 = df_NER.loc[df_NER["Year"] == 2020,]

In [8]:
df2021 = df_NER.loc[df_NER["Year"] == 2021,]

In [9]:
df2022 = df_NER.loc[df_NER["Year"] == 2022,]

In [10]:
df0 = df2020.merge(players_2020, how='left', on='Player')

In [11]:
df0 = df0.drop(['Year_y'], axis=1)
df0 = df0.rename(columns={'Year_x': 'Year'})

In [12]:
df1 = df2021.merge(players_2021, how='left', on='Player')

In [13]:
df1 = df1.drop(['Year_y'], axis=1)
df1 = df1.rename(columns={'Year_x': 'Year'})

In [14]:
df2 = df2022.merge(players_2022, how='left', on='Player')

In [15]:
df2 = df2.drop(['Year_y'], axis=1)
df2 = df2.rename(columns={'Year_x': 'Year'})

In [16]:
df_players = pd.concat([ df0, df1, df2], axis=0)

#### Club Name Cleaning

In the dictionary of clubs for NER, I added common nicknames of clubs which could be used. This section retransforms these names to the clubs proper name

In [17]:
df_players["Clubs"] = [l.replace("PSG", "Paris Saint-Germain") for l in df_players["Clubs"]]

In [18]:
df_players["Clubs"] = [l.replace("Reds", "Manchester United") for l in df_players["Clubs"]]
df_players["Clubs"] = [l.replace("Red Devils", "Manchester United") for l in df_players["Clubs"]]

In [19]:
df_players["Clubs"] = [l.replace("Blues", "Chelsea") for l in df_players["Clubs"]]

In [20]:
df_players["Clubs"] = [l.replace("Gunners", "Arsenal") for l in df_players["Clubs"]]

In [21]:
df_players["Clubs"] = [l.replace("Spurs", "Tottenham") for l in df_players["Clubs"]]

#### Finding the interested club

In [22]:
#Similiarty between words function to account for small differences in how cubs are spelt
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [23]:
#This allows to iterate through a list
from ast import literal_eval

In [24]:
#For each club in the clubs column, we check if the name is, or is similar to the name of the players club.
#If its not , we assume this club is interested in buying the player and add it to a new column which will contain the interested clubs.
n = 0
interested_club = []
for row in df_players["Clubs"] : 
    temp = []
    for club in literal_eval(row) : 
        if (similar(club, str(df_players.iloc[n,12])) < 0.7) & (club not in str(df_players.iloc[n,12]))  :
            temp.append(club)
    interested_club.append(temp)
    n += 1

In [25]:
df_players["Interested_clubs"] = interested_club

In [26]:
#df_players.iloc[0:50,[3,4,12,13]]

#### Checking for the Transfer

In [27]:
transfers = pd.read_csv("C:/Users/Titoran/Documents/Transfers Data/transfers.csv", index_col = [0])

In [28]:
#For each row, we check if the player has been involved in a transfer to one of the interested clubs. If yes, we consider the outcome to be True

Outcome = []
#n allows to keep track of which row we are iterating through
n = 0 
for player in df_players["Player"] :
    temp = 0 
    try : 
        for interested in df_players.iloc[n,13] : 
            for club in transfers.loc[(transfers["player_name"] == player), "club_name"] :
                if ((similar(club, str(interested)) > 0.7) | (club in str(interested)) | (str(interested) in club)):  
                    temp = 1
        if temp == 1 : 
            Outcome.append("True")
        else : 
            Outcome.append("False")
    except IndexError : 
            Outcome.append("False")
    n += 1

In [29]:
df_players["Outcome"] = Outcome

In [30]:
df_players

,Rumour,Year,Outlet,Clubs,Player,League,Player Link,Position,Age,Country,Join,Market Value,Club,Interested_clubs,Outcome
0,Newly-crowned Premier League champions Liverpo...,2020,"Tuttomercato, via Mail","['Liverpool', 'Napoli']",Kalidou Koulibaly,Serie A,https://www.transfermarkt.co.uk/kalidou-koulib...,Centre-Back,30.0,Senegal,2922.0,48.0,SSC Napoli,[Liverpool],False
1,Bayer Leverkusen coach Peter Bosz accepts that...,2020,Sun,"['Bayer Leverkusen', 'Chelsea', 'Real Madrid',...",Kai Havertz,Premier League,https://www.transfermarkt.co.uk/kai-havertz/pr...,Attacking Midfield,22.0,Germany,665.0,70.0,Chelsea,"[Bayer Leverkusen, Real Madrid, Manchester Uni...",False
2,Juventus director Fabio Paratici has played do...,2020,"Sky Sports Italia, via Metro","['Juventus', 'Manchester United', 'Arsenal', '...",Adrien Rabiot,Serie A,https://www.transfermarkt.co.uk/adrien-rabiot/...,Central Midfield,26.0,France,1096.0,30.0,Juventus,"[Manchester United, Arsenal, Everton]",False
3,The much rumoured midfielder swap deal involvi...,2020,Calciomercato,"['Barcelona', 'Juventus']",Miralem Pjanic,LaLiga,https://www.transfermarkt.co.uk/miralem-pjanic...,Central Midfield,31.0,Bosnia-Herzegovina,668.0,20.0,Barcelona,[Juventus],False
4,Real Valladolid want to extend Sandro Ramirez'...,2020,Liverpool Echo,"['Valladolid', 'Everton']",Sandro Ramirez,LaLiga,https://www.transfermarkt.co.uk/sandro-ramirez...,Centre-Forward,25.0,Spain,634.0,3.5,SD Huesca,"[Valladolid, Everton]",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Tottenham full-back Djed Spence, 22, is set to...",2022,Athletic - subscription required,"['Tottenham', 'Rennes']",Djed Spence,Championship,https://www.transfermarkt.co.uk/djed-spence/pr...,Right-Back,21.0,England,912.0,8.0,Middlesbrough,"[Tottenham, Rennes]",True
996,Leeds have enquired about signing Italy winger...,2022,Calciomercato,"['Leeds', 'Roma']",Nicolo Zaniolo,Serie A,https://www.transfermarkt.co.uk/nicolo-zaniolo...,Attacking Midfield,22.0,Italy,1461.0,33.0,AS Roma,[Leeds],False
997,A training ground bust-up between Joao Cancelo...,2022,Mail,"['Manchester City', 'Bayern Munich']",Joao Cancelo,Premier League,https://www.transfermarkt.co.uk/joao-cancelo/p...,Right-Back,28.0,Portugal,1059.0,65.0,Manchester City,[Bayern Munich],False
998,Norwich are close to signing 19-year-old Brazi...,2022,Express,"['Norwich', 'Arsenal']",Marquinhos,Ligue 1,https://www.transfermarkt.co.uk/marquinhos/pro...,Centre-Back,28.0,Brazil,3269.0,70.0,Paris Saint-Germain,"[Norwich, Arsenal]",True


In [31]:
#Checking the transfers which did happen
#df_players.loc[df_players["Outcome"] == "True",]

In [32]:
df_players.to_csv('Uncleaned_Data.csv', encoding='utf-8')